In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math

2024-02-06 23:28:40.691622: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 23:28:40.723962: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_file = "../train.tsv"
test_file = "../test.tsv"

train_data = pandas.read_csv(train_file, delimiter='\t').to_numpy()
train_data2 = np.empty((train_data.shape[0], train_data.shape[1]-2))
train_data2[:, 0:] = train_data[:, 2:]
train_data2[:,0:1] = train_data2[:, 0:1] * 22.17 / 4   #季節を正規化
train_data2[:,2:3] = train_data2[:, 2:3] * 17.93 / 12 #月を正規化
train_data2[:,3:4] = train_data2[:, 3:4] * 40.75 / 23 #時間を正規化
train_data2[:,5:6] = train_data2[:, 5:6] * -0.04 / 6.0   #曜日を正規化
train_data2[:,7:8] = train_data2[:, 7:8] * 14.33 / 4.0   #天気を正規化
train_data2[:,4:5] = train_data2[:, 4:5] * -2.29
train_data2[:,6:7] = train_data2[:, 6:7] * 1.17
train_data2[:,8:9] = train_data2[:, 8:9] * 45.12
train_data2[:,9:10] = train_data2[:, 9:10] * 44.70
train_data2[:,10:11] = train_data2[:, 10:11] * -28.86
train_data2[:,11:12] = train_data2[:, 11:12] * 8.54

train_data2 = train_data2#.astype(np.float32)
rnd = np.random.default_rng()
rnd.shuffle(train_data2,axis=0)

test_data = pandas.read_csv(test_file, delimiter='\t').to_numpy()
test_data2 = np.empty((test_data.shape[0], test_data.shape[1]-2))
test_data2[:, 0:] = test_data[:, 2:]
test_data2[:,0:1] = test_data2[:, 0:1] * 22.17 / 4   #季節を正規化
test_data2[:,2:3] = test_data2[:, 2:3] * 17.93 / 12 #月を正規化
test_data2[:,3:4] = test_data2[:, 3:4] * 40.75 / 23 #時間を正規化
test_data2[:,5:6] = test_data2[:, 5:6] * -0.04 / 6.0   #曜日を正規化
test_data2[:,7:8] = test_data2[:, 7:8] * 14.33 / 4.0   #天気を正規化
test_data2[:,4:5] = test_data2[:, 4:5] * -2.29
test_data2[:,6:7] = test_data2[:, 6:7] * 1.17
test_data2[:,8:9] = test_data2[:, 8:9] * 45.12
test_data2[:,9:10] = test_data2[:, 9:10] * 44.70
test_data2[:,10:11] = test_data2[:, 10:11] * -28.86
test_data2[:,11:12] = test_data2[:, 11:12] * 8.54
test_data2 = test_data2#.astype(np.float32)

#display(train_data2)
#display(test_data2)

x = train_data2[:, 12:]
x = x.reshape(x.shape[0], 1)
#print(x)
y = train_data2[:, :12]
y = y.reshape(y.shape[0], y.shape[1], 1)
#print(y)
test_y = test_data2.reshape(test_data2.shape[0], test_data2.shape[1], 1)
z = test_data2[:, :12]
z = z.reshape(z.shape[0], z.shape[1], 1)



In [3]:
osaka_file = "../test_local_osaka3.tsv"
osaka_data = pandas.read_csv(osaka_file, delimiter='\t').to_numpy()
osaka_data2 = np.empty((osaka_data.shape[0], osaka_data.shape[1]-2))
osaka_data2[:, 0:] = osaka_data[:, 2:]
osaka_data2[:,0:1] = osaka_data2[:, 0:1] * 22.17 / 4   #季節を正規化
osaka_data2[:,2:3] = osaka_data2[:, 2:3] * 17.93 / 12 #月を正規化
osaka_data2[:,3:4] = osaka_data2[:, 3:4] * 40.75 / 23 #時間を正規化
osaka_data2[:,5:6] = osaka_data2[:, 5:6] * -0.04 / 6.0   #曜日を正規化
osaka_data2[:,7:8] = osaka_data2[:, 7:8] * 14.33 / 4.0   #天気を正規化
osaka_data2[:,4:5] = osaka_data2[:, 4:5] * -2.29
osaka_data2[:,6:7] = osaka_data2[:, 6:7] * 1.17
osaka_data2[:,8:9] = osaka_data2[:, 8:9] * 45.12
osaka_data2[:,9:10] = osaka_data2[:, 9:10] * 44.70
osaka_data2[:,10:11] = osaka_data2[:, 10:11] * -28.86
osaka_data2[:,11:12] = osaka_data2[:, 11:12] * 8.54
osaka_data2 = osaka_data2.astype(np.float32)
osaka_y = osaka_data2.reshape(osaka_data2.shape[0], osaka_data2.shape[1], 1)
#display(osaka_data2)

In [4]:
length_of_sequence = y.shape[1]
#print(length_of_sequence)
in_out_neurons = 1 #入出力ノード数
n_hidden = 1024  #隠れ層 -> 数が多い程，学習モデルの複雑さが増加 #増やした

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(length_of_sequence, in_out_neurons), return_sequences=False))
model.add(Dense(in_out_neurons))
model.add(Activation("linear"))
optimizer = Adam(learning_rate=0.001)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["accuracy"])

2024-02-06 23:28:42.541944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 23:28:42.569610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 23:28:42.569665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 23:28:42.572184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 23:28:42.572235: I tensorflow/compile

KeyboardInterrupt: 

In [ ]:
#def build_model(hp):
#  import keras
#  model = keras.Sequential()
#  model.add(keras.layers.Dense(
#      hp.Choice('units', [8, 16, 32]),
#      activation='relu'))
#  model.add(keras.layers.Dense(1, activation='relu'))
#  model.compile(loss='mse')
#  return model

#tuner = kt.RandomSearch(
#    build_model,
#    objective='val_loss',
#    max_trials=5)

#tuner.search(train_data2, epochs=5, validation_data=(train_data2))
#best_model = tuner.get_best_models()[0]

In [ ]:
ES = EarlyStopping(mode='auto', patience=32)
model.fit(y, x,
          batch_size=16, #デフォは32
          epochs=512, #512epochで学習
          validation_split=0.01, #学習データの5%をvalidationに用いる
          callbacks=[ES]
          )

In [ ]:
predicted = model.predict(y,batch_size=32)
model.evaluate(y,x,batch_size=32)

In [ ]:
def check(i):
    import math
    if i< 0:
        return 0
    else:
        return math.floor(i)

In [ ]:
checked = np.array([check(i) for i in predicted]).reshape(predicted.shape[0],1)
r2 = 1 -  np.sum((checked - x)**2) / np.sum((x - np.mean(x))**2)
print(r2)

In [ ]:
plt.figure(figsize=(100,10))
plt.plot(range(0,len(checked)),checked, color="r", label="predict_data")
plt.plot(range(0, len(x)), x, color="b", label="row_data", alpha=0.5)
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(range(0,len(checked)),checked, color="r", label="predict_data")
plt.plot(range(0, len(x)), x, color="b", label="row_data", alpha=0.5)
plt.legend()
plt.show()

In [ ]:
predictedz = model.predict(z,batch_size=32)
checkedz = np.array([check(i) for i in predictedz]).reshape(predictedz.shape[0],1)

plt.figure(figsize=(10,5))
plt.plot(range(0,len(checkedz)),checkedz, color="r", label="predict_data")
plt.legend()
plt.show()

In [ ]:
p = np.hstack(checked)
np.savetxt("p.csv",p,delimiter=",",fmt="%s")

q = np.hstack(checkedz)
np.savetxt("q.csv",q,delimiter=",",fmt="%s")

In [ ]:
osaka_predicted = model.predict(osaka_y)
osaka_checked = np.array([check(i) for i in osaka_predicted]).reshape(osaka_predicted.shape[0],1)
plt.figure(figsize=(100,10))
plt.plot(range(0,len(osaka_checked)),osaka_checked, color="r", label="predict_data")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(range(0,len(osaka_checked)),osaka_checked, color="r", label="predict_data")
plt.legend()
plt.show()

In [ ]:
osaka_data3 = np.hstack((osaka_data,osaka_checked))
np.savetxt(".tsvってなに？.csv",osaka_data3,delimiter=",",fmt="%s")

In [ ]:
tokyo_file = "../test_local_tokyo.tsv"
tokyo_data = pandas.read_csv(tokyo_file, delimiter='\t').to_numpy()
tokyo_data2 = np.empty((tokyo_data.shape[0], tokyo_data.shape[1]-2))
tokyo_data2[:, 0:] = tokyo_data[:, 2:]
tokyo_data2[:,0:1] = tokyo_data2[:, 0:1] * 22.17 / 4   #季節を正規化
tokyo_data2[:,2:3] = tokyo_data2[:, 2:3] * 17.93 / 12 #月を正規化
tokyo_data2[:,3:4] = tokyo_data2[:, 3:4] * 40.75 / 23 #時間を正規化
tokyo_data2[:,5:6] = tokyo_data2[:, 5:6] * -0.04 / 6.0   #曜日を正規化
tokyo_data2[:,7:8] = tokyo_data2[:, 7:8] * 14.33 / 4.0   #天気を正規化
tokyo_data2[:,4:5] = tokyo_data2[:, 4:5] * -2.29
tokyo_data2[:,6:7] = tokyo_data2[:, 6:7] * 1.17
tokyo_data2[:,8:9] = tokyo_data2[:, 8:9] * 45.12
tokyo_data2[:,9:10] = tokyo_data2[:, 9:10] * 44.70
tokyo_data2[:,10:11] = tokyo_data2[:, 10:11] * -28.86
tokyo_data2[:,11:12] = tokyo_data2[:, 11:12] * 8.54
tokyo_data2 = tokyo_data2.astype(np.float32)
tokyo_y = tokyo_data2.reshape(tokyo_data2.shape[0], tokyo_data2.shape[1], 1)

In [ ]:
tokyo_predicted = model.predict(tokyo_y)
tokyo_checked = np.array([check(i) for i in tokyo_predicted]).reshape(tokyo_predicted.shape[0],1)
plt.figure(figsize=(100,10))
plt.plot(range(0,len(tokyo_checked)),tokyo_checked, color="r", label="predict_data")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(range(0,len(tokyo_checked)),tokyo_checked, color="r", label="predict_data")
plt.legend()
plt.show()

tokyo_data3 = np.hstack((tokyo_data,tokyo_checked))
np.savetxt("tokyo.csv",tokyo_data3,delimiter=",",fmt="%s")